# U.S. Patent Phrase to Phrase Matching
Help Identify Similar Phrases in U.S. Patents


### On Kaggle

In [ ]:
from fastai.imports import *

In [ ]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
path = (Path('../input/us-patent-phrase-to-phrase-matching') if iskaggle
    else Path.home()/'data'/'us-patent-phrase-to-phrase-matching')
path.ls()

In [ ]:
df = pd.read_csv(path/'train.csv')
df

In [ ]:
df.describe(include='object')

In [ ]:
eval_df = pd.read_csv(path/'test.csv')
len(eval_df)

In [ ]:
eval_df.head()

In [ ]:
df.target.value_counts()

In [ ]:
df.anchor.value_counts()

In [ ]:
df.context.value_counts()

In [ ]:
df.score.hist();

In [ ]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

## Tokenization

In [ ]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

In [ ]:
model_nm = '../input/debertav3base/deberta-v3-base'

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
def tok_func(x): return tokz(x["input"])

In [ ]:
tok_func(ds[0])

In [ ]:
tokz.all_special_tokens


In [ ]:
inps = "anchor","target","context"
tok_ds = ds.map(tok_func, batched=True)
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [ ]:
row = tok_ds[0]
row['input']

In [ ]:
row

In [ ]:
row['input_ids']

## Test and validation sets

In [ ]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

### Validation set

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

### Test set

In [ ]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

In [ ]:
def corr_d(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

## Training

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
lr = 8e-5
bs = 128
epochs = 6
wd = 0.01

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
trainer.train();

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

In [ ]:
preds = np.clip(preds, 0, 1)

In [ ]:
preds

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds.flatten()
})

submission.to_csv('submission.csv', index=False)